## Mount Google Drive for saving model weights later

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Get dataset from github

In [2]:
!git clone https://github.com/hpfield/ads_dataset.git # Comes resized to 224x224

Cloning into 'ads_dataset'...
remote: Enumerating objects: 15837, done.
remote: Total 15837 (delta 0), reused 0 (delta 0), pack-reused 15837
Receiving objects: 100% (15837/15837), 721.26 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [3]:
import os
import shutil
import random
import pandas as pd
import cv2
import numpy as np
from tqdm import tqdm

## Segment the dataset into train, test

In [4]:

# Set the seed for reproducibility
random.seed(42)

# Set the path to the original dataset
original_dataset_dir = '/content/ads_dataset/uob_image_set_resized/'

# Set the base directory for the new dataset
base_dir = '/content/ads_dataset/YOLOv5_train_test_classification/'

# Remove directories if exist
if os.path.exists(base_dir):
    shutil.rmtree(base_dir)
os.mkdir(base_dir)
# Create the directories

# Create the train directory
train_dir = os.path.join(base_dir, 'train')

if not os.path.exists(train_dir):
    os.mkdir(train_dir)

# Create the test directory
test_dir = os.path.join(base_dir, 'test')

if not os.path.exists(test_dir):
    os.mkdir(test_dir)



In [5]:
# Create the directories for each class using the original dataset directory
# each folder is it's own class

# Get the folder names from the original dataset directory

classes = os.listdir(original_dataset_dir)

# Remove hidden files
classes = [c for c in classes if not c.startswith('.')]

# Create the directories for each class in the train and test directories

for class_name in classes:
    # Create the train directory for the class
    train_class_dir = os.path.join(train_dir, class_name)
    
    if not os.path.exists(train_class_dir):
        os.mkdir(train_class_dir)
        
    # Create the test directory for the class
    test_class_dir = os.path.join(test_dir, class_name)
    
    if not os.path.exists(test_class_dir):
        os.mkdir(test_class_dir)



In [6]:
# Copy the images into the train and test directories

# Get the list of images for each class and store them in a dataframe

# Create a dataframe to store the image names and their class
dataset = pd.DataFrame(columns=['image_name', 'class'])

# Loop through each class
for class_name in classes:
    # Get the list of images for the class
    images = os.listdir(os.path.join(original_dataset_dir, class_name))
    for image in images:
        # Add the image name and class to the dataframe using pd.concat
        dataset = pd.concat([dataset, pd.DataFrame({'image_name': [image], 'class': [class_name]})], ignore_index=True)

In [7]:
# Split the dataset into train and test
# Use a 80/20 split
# Ensure that each class has at least one example in both the train and test sets

# Create a dataframe to store the train images
train = pd.DataFrame(columns=['image_name', 'class'])

# Create a dataframe to store the test images
test = pd.DataFrame(columns=['image_name', 'class'])

# Loop through each class
for class_name in classes:
    # Get the list of images for the class
    images = dataset[dataset['class'] == class_name]
    
    # Get the number of images for the class
    num_images = len(images)
    
    # Get the number of images to use for the test set
    num_test_images = int(num_images * 0.2)
    
    # Get the number of images to use for the train set
    num_train_images = num_images - num_test_images
    
    # Get the images to use for the train set
    train_images = images.sample(n=num_train_images, random_state=42)
    
    # Get the images to use for the test set
    test_images = images.drop(train_images.index)

    if len(test_images) == 0:
        # If there are no images in the test set, move one image from the train set to the test set
        test_images = train_images.sample(n=1, random_state=42)
        train_images = train_images.drop(test_images.index)
        
    # Add the images to the train and test dataframes
    train = pd.concat([train, train_images], ignore_index=True)
    test = pd.concat([test, test_images], ignore_index=True)

In [8]:
# Copy the images into the train and test directories

# Loop through each class
for class_name in classes:
    # Get the list of images for the class
    train_images = train[train['class'] == class_name]['image_name'].tolist()
    test_images = test[test['class'] == class_name]['image_name'].tolist()

    # Check if the train_images and test_images lists are empty
    if len(train_images) == 0:
        print('No train images for class: {}'.format(class_name))

    if len(test_images) == 0:
        print('No test images for class: {}'.format(class_name))
    
    # Copy the images into the train and test directories
    for image in train_images:
        src = os.path.join(original_dataset_dir, class_name, image)
        dst = os.path.join(train_dir, class_name, image)
        shutil.copyfile(src, dst)
        
    for image in test_images:
        src = os.path.join(original_dataset_dir, class_name, image)
        dst = os.path.join(test_dir, class_name, image)
        shutil.copyfile(src, dst)

In [9]:
# Check if any of the folders in the train and test directories are empty

# Loop through each class
for class_name in classes:
    # Get the list of images for the class
    train_images = os.listdir(os.path.join(train_dir, class_name))
    test_images = os.listdir(os.path.join(test_dir, class_name))
    
    # Check if the train and test directories are empty
    if len(train_images) == 0:
        print('The train directory for the class {} is empty'.format(class_name))
    if len(test_images) == 0:
        print('The test directory for the class {} is empty'.format(class_name))

## Setup clearml

In [10]:
!pip install clearml -q
%env CLEARML_WEB_HOST=https://app.clear.ml
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
%env CLEARML_API_ACCESS_KEY=LBZ150QG3IKW0619OSAO
%env CLEARML_API_SECRET_KEY=jL17F7bprtetI3heJnSpyGVxREiw9csdJdt2AEUjTcWiAJvzbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.3 MB/s eta 0:00:00
env: CLEARML_WEB_HOST=https://app.clear.ml
env: CLEARML_API_HOST=https://api.clear.ml
env: CLEARML_FILES_HOST=https://files.clear.ml
env: CLEARML_API_ACCESS_KEY=LBZ150QG3IKW0619OSAO
env: CLEARML_API_SECRET_KEY=jL17F7bprtetI3heJnSpyGVxREiw9csdJdt2AEUjTcWiAJvzbt


## Download YOLOv5

In [11]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-145-g94714fe Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 24.2/78.2 GB disk)


### Configure model and training run with ClearML

In [24]:
project_name = 'snap-vision'
# Update the below variables to reflect the experiment details
num_epochs = '100' 
image_size = '224'
augmentations = 'no-augmentations' # Do not use spaces
experiment_name = num_epochs + '-epochs,' + image_size + ',' + augmentations # Change this line to reflect the current training run

In [25]:
# insert lines of code into train.py for clearml

# Read the existing contents of the file
with open("classify/train.py", "r") as file:
    existing_lines = file.readlines()

# List of lines to be inserted at the beginning
lines_to_insert = ["from clearml import Task", "task = Task.init(project_name='"+project_name+"', task_name='"+experiment_name+"')"]

file.close()

flag = lines_to_insert[0] + '\n'
# if lines_to_insert[0] already exists in existing_lines, remove it and the line immediately folowing it
for i in range(len(existing_lines)-2):
    if existing_lines[i] == flag:
        existing_lines.pop(i)
        existing_lines.pop(i)

# Insert the lines at the beginning
with open("classify/train.py", "w") as file:
    # Write the lines to be inserted followed by a newline character
    for line in lines_to_insert:
        file.write(line + "\n")

    # Write the existing contents of the file after the inserted lines
    for line in existing_lines:
        file.write(line)

# Close the file
file.close()                    


## Train model

In [ ]:
# Train YOLOv5s Classification on fashion images
!python classify/train.py --model yolov5s-cls.pt --data '/content/ads_dataset/YOLOv5_train_test_classification/' --epochs {num_epochs} --img 224 --cache

ClearML Task: created new task id=9fa3295e1b2d472981004e35650bb44f
ClearML results page: https://app.clear.ml/projects/bfdb6aab40524325966d5ec5b0e06572/experiments/9fa3295e1b2d472981004e35650bb44f/output/log
classify/train: model=yolov5s-cls.pt, data=/content/ads_dataset/YOLOv5_train_test_classification/, epochs=100, batch_size=64, imgsz=224, nosave=False, cache=ram, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=True, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_rank=-1
github: up to date with https://github.com/ultralytics/yolov5 ✅
requirements: /content/requirements.txt not found, check failed.
YOLOv5 🚀 v7.0-145-g94714fe Python-3.9.16 torch-2.0.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

TensorBoard: Start with 'tensorboard --logdir runs/train-cls', view at http://localhost:6006/
albumentations: RandomResizedCrop(p=1.0, height=224, width=224, scale=(0.08, 1.0), ratio=(0.75, 1.333333333

## Copy weights to Drive for later use

In [ ]:
weights_source = '/content/yolov5/runs/train-cls/exp/weights/best.pt'
weights_dest = '/content/drive/MyDrive/Applied_Data_Science/yolov5/classification/weights/' + experiment_name + '/'
if not os.path.exists(weights_dest):
        os.makedirs(weights_dest)
shutil.copy(weights_source, weights_dest)

To change the colour of the label text, go to classify/predict.py line 147

To update the weights to reflect a new training run, change the exp in the weights argument